# Perceptron

O perceptron, mais especificamente o Perceptron de Rosenblatt, é um dispositivo computacional capaz de classificar padrões linearmente separáveis. Formalmente, a saída do perceptron é calculada por:

$$y = \text{signum}(w \cdot x )$$

tal que $x = \{+1, x_1, x_2, \dots, x_m\}$ é um vetor de entrada com a primeira entrada fixada em +1 e $w = \{w_0, w_1, w_2, \dots, w_m \}$ é o vetor de pesos sinápticos. Signum é uma função quantizadora que mapeia valores positivos para 1 e valores negativos para -1:

$$ \text{signum}(v) =
  \begin{cases}
    -1       & \quad \text{if } v \leq 0\\
    1  & \quad \text{if } v \gt 0
  \end{cases}
$$

Logo, o perceptron classifica o vetor de entrada $x$ em uma entre duas classes possíveis.

In [ ]:
%matplotlib inline
from sklearn.datasets import load_iris
import matplotlib.pyplot as plt
import numpy as np
import random

## Funções auxiliares

Estas funções servem para ajudar a analisar o funcionamento do perceptron. Em especial, a função acc_perceptron abaixo retorna a acurácia do perceptron em um conjunto de dados X e rótulos Y. A acurácia é dada por:

$$acc = \frac{\text{acertos}}{\text{acertos}+\text{erros}}$$

Desta forma a acurácia é um número que vai de $0$ a $1$ e indica a proporção de acertos em relação ao número total de exemplos.

In [ ]:
#Plota as características feature_idxs (lista com 2 indíces de características)
#X é um vetor onde todos os elementos antes do sep_idxs são da classe 1 e 
#todos de sep_idxs + 1 até o fim são da classe 2.
def plot_features(X, sep_idx, feature_idxs):
    c1 = 'red'
    c2 = 'green'
    
    f1_a = []
    f2_a = []
    f1_b = []
    f2_b = []
    
    for i in X[:sep_idx]:
        f1_a.append(i[feature_idxs[0]])
        f2_a.append(i[feature_idxs[1]])
    
    for i in X[sep_idx + 1:]:
        f1_b.append(i[feature_idxs[0]])
        f2_b.append(i[feature_idxs[1]])   
        
    plt.scatter(f1_a, f2_a, c=c1)
    plt.scatter(f1_b, f2_b, c=c2)

#Retorna a acurácia do perceptron. X e Y são vetores paralelos e w é o vetor de pesos.
def acc_perceptron(X, Y, w):
    hits = float(0)
    misses = float(0)
    for n in xrange(len(X)):
        x_n = X[n]
        d_n = d(Y, n)
        y_n = eval_perceptron(x_n, w)
        if (d_n - y_n) == 0:
            hits+=1
        else:
            misses+=1
            
    return hits/(hits+misses)
    

## Implementação do Perceptron

As funções **signum** e **eval_perceptron** estão descritas no início deste caderno. A função **d** indica a saída desejada do vetor indexado por *idx* em *targets*. Nesta implementação, a classe com rótulo 0 é considerada positiva, enquanto as classes com outros rótulos são consideradas negativas.

### Perceptron Convergence

É o algoritmo online de aprendizagem do perceptron. Recebe uma matriz de características X, onde cada linha é um exemplo e cada coluna uma característica; um vetor de rótulos Y, paralelo ao vetor X; $0 \le \eta \leq 1$ (eta), a taxa de aprendizagem; e *shuffle*, um parametro que indica se é necessário embaralhar o vetor X e Y paralelamente antes do treino.

Este algoritmo é equivalente à descida do gradiente em relação à função de threshold. No entanto, vamos ver a razão disso mais pra frente. A idéia é atualizar os pesos apenas se houver um erro na predição. a atualização é realizada da seguinte forma:

$$w(n+1) = w(n) + \eta (d(Y,n) - y(n)) x(n)$$

Tal que $w(n)$ é o vetor de pesos na iteração $n$, $d(Y,n)$ é o valor desejado do perceptron na iteração $n$, $y(n)$ é a saída obtida pelo perceptron na iteração $n$ e $x(n)$ é o vetor de entrada na interação $n$.


In [ ]:
#Função signum, que quantiza a saída do perceptron pra 1 se x for positivo ou -1, caso contrário.
def signum(x):
    return 1 if x > 0 else -1

#Função d que indica a saída desejada. Note a classe 0 está mapeada para 1, enquanto 
#as outras estão mapeadas para -1
def d(targets, idx):
    return 1 if targets[idx] == 0 else -1

#Função do perceptron threshold com função signum
def eval_perceptron(x, w):
    return signum(np.dot(w,x))

#convergência do perceptron.
def perceptron_convergence(X, Y, eta, shuffle=True,w_inicial=None):
    if shuffle:
        s= np.arange(len(X))
        np.random.shuffle(s)
        X = X[s]
        Y = Y[s]
    
    #inicializar vetor de pesos com todos os valores em 0 caso um w_inicial não seja passado.
    w = w_inicial if w_inicial is not None else np.zeros(X.shape[1])
    
    for k in xrange(1):
        for n in xrange(len(X)):
            x_n = X[n]
            y_n = eval_perceptron(x_n, w)
            w = w + eta * (d(Y, n) - y_n)* x_n
        
    return w

## Bancada de Testes do Perceptron

Aqui é apresentada uma bancada de testes para o perceptron em um problema conhecido por ser linearmente separável.

A base de dados usada é a Iris (do sklearn).

### Traçando a Fronteira de Decisão

Note que $y = w \cdot x \Leftrightarrow y = w_0x_0 + w_1x_1 + w_2x_2$, no caso que $m = 2$.

Sabemos que a fronteira de decisão é a reta que passa exatamente quando $wx = 0$. Assim, sabemos que

$$w_0x_0 + w_1x_1 + w_2x_2 = 0$$

Isolando $x_1$, temos

$$x_1 = \frac{-w_0 - w_2x_2}{w_1}$$

Assim, a reta cruza o eixo $x_1$ ($x_2 = 0$):

$$Xx_1 = \frac{-w0}{w1}$$

da mesma forma, a reta cruza o eixo $x_2$ ($x_1 = 0$)

$$Xx_2 = \frac{-w0}{w2}$$

Assim, sabemos que os pontos $P1 = \left(\frac{-w0}{w1}, 0 \right)$ e $P2 = \left(0, \frac{-w0}{w2} \right)$

Podemos usar essa informação para montar a equação da reta correspondente. Primeiro basta encontrar o coeficiente angular usando P1 e P2:

$$m = \frac{P2y - P1y}{P2x-P2y}$$

Tal que $x$ é a coordenada em $x_1$ e $y$ é a coordenada em $x_2$. Substituindo e expandindo:

$$m = \frac{-w1}{w2} $$

Desta forma, $x_2$ em função de $x_1$ pode ser escrito como:

$$x_2 = m x_1 + Xx_2 $$

$$x_2 = \frac{-w_1}{w_2}x_1 + \frac{-w_0}{w_2}$$

In [ ]:
dataset = load_iris()

X = dataset.data
Y = dataset.target

#adicionar a coluna de 1's (bias)
X = np.insert(X, 0, np.array([1] * len(X)), axis=1)

#usar os 100 primeiros exemplos e 2 primeiras características (e o bias na primeira coluna).
#Escolhi os 100 primeiros pq os 50 primeiros são da classe 0 e os próximos 50 são da classe 1.
X = X[:100,:3]
#recuperar também os rótulos dos 100 primeiros exemplos
Y = Y[:100]

#Treinar o perceptron
w = perceptron_convergence(X, Y, eta=0.1)
print('vetor de pesos apos o treino: %s' % str(w))

#plotar as features!
plot_features(X, 50, [1,2])

#plotar a fronteira de decisão
xdb = np.linspace(np.min(X[:,1]),np.max(X[:,1]),20)
ydb = map(lambda x: (-w[1]/w[2])*x + (-w[0]/w[2]), xdb)
plt.plot(xdb, ydb)

print('Acurácia: %.2f' % acc_perceptron(X, Y, w))


In [ ]:
w = perceptron_convergence(X, Y, eta=0.1, w_inicial=w)
#plotar as features!
plot_features(X, 50, [1,2])

#plotar a fronteira de decisão
xdb = np.linspace(np.min(X[:,1]),np.max(X[:,1]),20)
ydb = map(lambda x: (-w[1]/w[2])*x + (-w[0]/w[2]), xdb)
plt.plot(xdb, ydb)

print('Acurácia: %.2f' % acc_perceptron(X, Y, w))

## Execução do Perceptron (passo-a-passo)

Veja a execução do perceptron passo-a-passo. Primeiramente execute a célula logo abaixo dessa. Isso irá executar o treino do perceptron e retornar todas as atualizações realizadas.

In [ ]:
def perceptron_convergence_steps(X, Y, eta, shuffle=True):
    #inicializar vetor de pesos com todos os valores em 0
    if shuffle:
        s= np.arange(len(X))
        np.random.shuffle(s)
        X = X[s]
        Y = Y[s]
        
    w = np.zeros(X.shape[1])
    all_w = [w]
    for n in xrange(len(X)):
        x_n = X[n]
        y_n = eval_perceptron(x_n, w)
        w = w + eta * (d(Y, n) - y_n)* x_n
        if (d(Y,n) - y_n) != 0:
            all_w.append(w)
                

    return all_w
                
dataset = load_iris()

X = dataset.data
Y = dataset.target

#adicionar a coluna de 1's (bias)
X = np.insert(X, 0, np.array([1] * len(X)), axis=1)

X = X[:100,:3]
Y = Y[:100]

min_x = np.min(X[:,1])
max_x = np.max(X[:,1])

ws = perceptron_convergence_steps(X, Y, 0.001)
i = 0
print('Foram executados %d passos de atualização dos pesos' % len(ws))

Após executar a célula acima uma vez, execute a célula abaixo várias vezes para ver como a fronteira de decisão muda conforme os pesos são atualizados!

In [ ]:
#Execute esta célula e veja as iterações do algoritmo de treinamento do perceptron.
print('passo %d' % i)
w = ws[i]

plot_features(X, 50, [1,2])

xdb = np.linspace(min_x, max_x,20)

ydb = map(lambda x: (-w[1]/w[2])*x + (-w[0]/w[2]), xdb)

plt.plot(xdb, ydb)
plt.xlim(min_x,max_x)

i+=1
    

## Convergência do *Perceptron* em Lote (*Batch*)

In [ ]:
def perceptron_convergence_batch(X, Y, eta, shuffle=True,w_inicial=None):
    if shuffle:
        s= np.arange(len(X))
        np.random.shuffle(s)
        X = X[s]
        Y = Y[s]
    
    #inicializar vetor de pesos com todos os valores em 0 caso um w_inicial não seja passado.
    w = w_inicial if w_inicial is not None else np.zeros(X.shape[1])
    w_deltas = []
    for k in xrange(1):
        for n in xrange(len(X)):
            x_n = X[n]
            y_n = eval_perceptron(x_n, w)
            w_deltas.append(eta * (d(Y, n) - y_n)* x_n)
    
    w_deltas = np.array(w_deltas)
        
    return w + np.mean(w_deltas, axis=0)

dataset = load_iris()

X = dataset.data
Y = dataset.target

#adicionar a coluna de 1's (bias)
X = np.insert(X, 0, np.array([1] * len(X)), axis=1)

#usar os 100 primeiros exemplos e 2 primeiras características (e o bias na primeira coluna).
#Escolhi os 100 primeiros pq os 50 primeiros são da classe 0 e os próximos 50 são da classe 1.
X = X[:100,:3]
#recuperar também os rótulos dos 100 primeiros exemplos
Y = Y[:100]

#Treinar o perceptron
w = perceptron_convergence_batch(X, Y, eta=0.01)
print('vetor de pesos apos o treino: %s' % str(w))

#plotar as features!
plot_features(X, 50, [1,2])

#plotar a fronteira de decisão
xdb = np.linspace(np.min(X[:,1]),np.max(X[:,1]),20)
ydb = map(lambda x: (-w[1]/w[2])*x + (-w[0]/w[2]), xdb)
plt.plot(xdb, ydb)

print('Acurácia: %.2f' % acc_perceptron(X, Y, w))


In [ ]:
w = perceptron_convergence_batch(X, Y, eta=0.01, w_inicial=w)
print('vetor de pesos apos o treino: %s' % str(w))
#plotar as features!
plot_features(X, 50, [1,2])

#plotar a fronteira de decisão
xdb = np.linspace(np.min(X[:,1]),np.max(X[:,1]),20)
ydb = map(lambda x: (-w[1]/w[2])*x + (-w[0]/w[2]), xdb)
plt.plot(xdb, ydb)

print('Acurácia: %.2f' % acc_perceptron(X, Y, w))